In [1]:
import os

import numpy as np
import pandas as pd

from pandas.tseries.frequencies import to_offset

In [2]:
data_dir = "./Data/"
os.listdir(data_dir)

['PricesFile1.csv',
 'NG_ImpliedVols2020.csv',
 'NG_ImpliedVols2022.csv',
 'NG_ImpliedVols2021.csv',
 'WTI_ImpliedVols2022.csv',
 'WTI_ImpliedVols2020.csv',
 'WTI_ImpliedVols2021.csv',
 'InterestRates.csv']

In [3]:
prices_df = pd.read_csv(data_dir+"PricesFile1.csv")

prices_df.pricedate = pd.to_datetime(prices_df.pricedate)
prices_df.delivdate = pd.to_datetime(prices_df.delivdate)

prices_df = prices_df.loc[(prices_df.pricedate < prices_df.delivdate)]

tuples = list(zip(*[prices_df.pricedate, prices_df.delivdate]))
index = pd.MultiIndex.from_tuples(tuples, names=["t", "T"])

prices_df.set_index(index, inplace=True)
prices_df.drop(columns=["pricedate", "delivdate"], inplace=True)

In [4]:
NG_prices_df = prices_df.groupby("priceindex").get_group("NYMEX Natural Gas")
NG_prices_df = NG_prices_df.loc[NG_prices_df.price > 0]

In [5]:
NG20_df = pd.read_csv(data_dir+"NG_ImpliedVols2020.csv")
NG21_df = pd.read_csv(data_dir+"NG_ImpliedVols2021.csv")
NG22_df = pd.read_csv(data_dir+"NG_ImpliedVols2022.csv")

frames = [NG20_df, NG21_df, NG22_df]
NG_IV_df = pd.concat(frames)

In [6]:
NG_IV_df.volatilitydate = pd.to_datetime(NG_IV_df.volatilitydate)
NG_IV_df.begtime = pd.to_datetime(NG_IV_df.begtime)

NG_IV_df = NG_IV_df.loc[(NG_IV_df.volatilitydate < NG_IV_df.begtime)]

tuples = list(zip(*[NG_IV_df.volatilitydate, NG_IV_df.begtime]))
index = pd.MultiIndex.from_tuples(tuples, names=["t", "T"])

NG_IV_df.set_index(index, inplace=True)
NG_IV_df.drop(columns=["volatilitydate", "begtime"], inplace=True)

In [7]:
NG_df = NG_IV_df.merge(NG_prices_df, how="inner", left_index=True, right_index=True)


In [8]:
IR_df = pd.read_csv(data_dir+"InterestRates.csv")

In [9]:
a = [to_offset(i) for i in IR_df.maturity]
b = [pd.Timestamp(i) for i in IR_df.pricedate]
c = [b[i] + a[i] for i in range(len(a))]
IR_df.maturity = c
IR_df.pricedate = pd.to_datetime(IR_df.pricedate)
IR_df.maturity = pd.to_datetime(IR_df.maturity)
IR_df.maturity = IR_df.maturity+to_offset("1D")

In [10]:
tuples = list(zip(*[IR_df.pricedate, IR_df.maturity]))
index = pd.MultiIndex.from_tuples(tuples, names=["t", "T"])

IR_df.set_index(index, inplace=True)
IR_df.drop(columns=["pricedate", "maturity"], inplace=True)

In [ ]:
NG_df = NG_df.merge(IR_df, how="inner", left_index=True, right_index=True)

In [ ]:
NG_df.drop(columns=["volatilityindex", "priceindex"], inplace=True)
NG_df.reset_indexe()
NG_df